In [ ]:
# OpenIntro Statistics Ch 1 + Ch 2: Data Understanding & Summarizing
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Load built-in Titanic dataset (no download needed!)
df = sns.load_dataset('titanic')

print("Dataset loaded successfully!")
print(f"\nShape: {df.shape[0]} rows × {df.shape[1]} columns")
print("\n First 3 rows:")
display(df.head(3))

print("\n Variable Types (OpenIntro Ch 1):")
print(df.dtypes)